# Demo - EV SFR Provision

ANDES case: ``ieee39_evsfr.xlsx``

Source data: ``ieee39_ev_s.csv``, ``ieee39_ev_f.csv``, ``ieee39_ev_fict.csv``

Output data: Source data are cut to the first two minutes as ``ieee39_ev_s_out_short.csv``, ``ieee39_ev_f_out_short.csv``, ``ieee39_ev_fict_out_short.csv``.

Then the emulated measured bus data ``BusFreq`` output are added in the file ``ieee39_ev_s_out_short_emu.csv``, ``ieee39_ev_f_out_short_emu.csv``, ``ieee39_ev_fict_out_short_emu.csv``.

Three scenarios are considered in this demo:
1. ``ieee39_ev_s_out.csv``: EVs SFR is not provisioned.
2. ``ieee39_ev_f_out.csv``: EVs SFR is provisioned with no charging time constraints.
3. ``ieee39_ev_fict_out.csv``: EVs SFR is provisioned with charging time constraints.

ANDES case file ``ieee39_evsfr.xlsx`` and output file ``ieee39_ev_fict_out_short_emu.csv`` can be used in AGVis to visualize the results.

In [1]:
import numpy as np
import pandas as pd
import scipy.signal as sig

import matplotlib.pyplot as plt
plt.rcParams.update(plt.rcParamsDefault)

import os

import andes

In [2]:
andes.__version__

'1.8.7.post3+g4f0a0b9a'

In [3]:
cwd = os.getcwd()

In [4]:
# Construct the file paths relative to the current working directory ev_s_path
ev_s_path = os.path.join(cwd, '../data/source/ieee39_ev_s_out.csv')
ev_f_path = os.path.join(cwd, '../data/source/ieee39_ev_f_out.csv')
ev_fict_path = os.path.join(cwd, '../data/source/ieee39_ev_fict_out.csv')

# Read the CSV files into pandas dataframes
dfs_read = pd.read_csv(ev_s_path)
dff_read = pd.read_csv(ev_f_path)
dfict_read = pd.read_csv(ev_fict_path)

In [5]:
evout_path = os.path.join(cwd, '../data/output/ieee39_evsfr_out.csv')
evout = pd.read_csv(evout_path)

In [6]:
ss = andes.load(os.path.join(cwd, '../data/case/ieee39_evsfr.xlsx'))

In [7]:
dff = dff_read[dff_read['Time [s]'] <=120]
dfs = dfs_read[dfs_read['Time [s]'] <= 120]
dfict = dfict_read[dfict_read['Time [s]'] <= 120]

In [8]:
dfs.to_csv(os.path.join(cwd, '../data/output/ieee39_ev_s_out_short.csv'), index=False)
dff.to_csv(os.path.join(cwd, '../data/output/ieee39_ev_f_out_short.csv'), index=False)
dfict.to_csv(os.path.join(cwd, '../data/output/ieee39_ev_fict_out_short.csv'), index=False)

In [9]:
dfict_emu = pd.DataFrame(data=np.zeros((dfict.shape[0], evout.shape[1])), columns=evout.columns.tolist())
dfict_emu['Time [s]'] = dfict['Time [s]']

t = dfict_emu['Time [s]'].values

In [10]:
aBus_cols = dff.filter(like='a Bus').columns.tolist()
f_cols = [f'f BusFreq {n+1}' for n in range(ss.Bus.n)]

In [11]:
# Create the transfer function object
Lag = sig.lti([0, 1], [0.2, 1])
Washout = sig.lti([1 / 2 / np.pi / ss.config.freq, 0], [0.02, 1])

for n in range(1, 40):
    a = dff[f'a Bus {n}'].values
    # Simulate the response to the input signal
    t, y, _ = sig.lsim2(Lag, a, t)
    t, freq, _ = sig.lsim2(Washout, y, t)
    dfict_emu[f'f BusFreq {n}'] = freq + 1
    dfict_emu[f'WO_y BusFreq {n}'] = freq

/Users/jinningwang/Documents/work/mambaforge/envs/demo/lib/python3.9/site-packages/scipy/signal/_filter_design.py:1746: BadCoefficients: Badly conditioned filter coefficients (numerator): the results may be meaningless
  warnings.warn("Badly conditioned filter coefficients (numerator): the "


In [12]:
dfict_emu_path = os.path.join(cwd, '../data/output/ieee39_ev_fict_out_short_emu.csv')
dfict_emu.to_csv(dfict_emu_path, index=False)